<a href="https://colab.research.google.com/github/sophiewagner7/its-too-nice-out-to-take-a-cab/blob/main/notebooks/pre_process_taxi_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process Taxi Data

Moacir P. de Sá Pereira

This notebook reduces the size of our yellow taxi and high-volume for-hire vehicle (Uber, Lyft) data, downloaded from the [Taxi and Limousine Commission](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). We drop columns that will not be important to us, rename the columns so that both yellow cab and FHVHV data will have the same column names, save `license`, which indicates which company operates the FHVHV trip in question. The codes convert to companies like this:

Code | Company
----|----
HV0002|Juno
HV0003|Uber
HV0004|Via
HV0005|Lyft

Additionally, we use the taxi zone lookup table to reduce our trip data only to trips that originate and terminate in Manhattan. This has a substantial effect on the FHVHV data, but little effect on the yellow cab data.

The files are then saved as parquet files in a `processed_files` folder.

Next, the processed files are concatenated to make yearly files, saved in `concatenated_files`.

## Imports

In [10]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Set Constants

In [11]:
root_path = "./drive/MyDrive/taxi-data"

zones_df = pd.read_csv(f"{root_path}/taxi_zone_lookup.csv")
manhattan_zones = list(zones_df[zones_df.Borough == "Manhattan"].LocationID)

yellow_columns_to_keep = {
    "tpep_pickup_datetime": "pickup_datetime",
    "tpep_dropoff_datetime": "dropoff_datetime",
    "trip_distance": "trip_distance",
    "PULocationID": "pickup_zone",
    "DOLocationID": "dropoff_zone",
    "fare_amount": "fare_amount",
    "tip_amount": "tip_amount"
}

fhvhv_columns_to_keep = {
    "pickup_datetime": "pickup_datetime",
    "dropoff_datetime": "dropoff_datetime",
    "trip_miles": "trip_distance",
    "PULocationID": "pickup_zone",
    "DOLocationID": "dropoff_zone",
    "base_passenger_fare": "fare_amount",
    "tips": "tip_amount",
    "hvfhs_license_num": "license",
}

license_relabel = {
    "yellow": "yellow",
    "HV0002": "juno",
    "HV0003": "uber",
    "HV0004": "via",
    "HV0005": "lyft"
}

## Pre-Processing Functions

In [12]:
def process_file(trip = "fhvhv", file_name = "fhvhv_tripdata_2021-05.parquet"):
  path = f"{root_path}/original_files/{file_name}"
  if trip == "yellow":
    columns = yellow_columns_to_keep
  else:
    columns = fhvhv_columns_to_keep
  df = pd.read_parquet(path)
  df = df[columns.keys()]
  df = df.rename(columns=columns)
  df = df[df.pickup_zone.isin(manhattan_zones) & df.dropoff_zone.isin(manhattan_zones)]
  df.to_parquet(f"{root_path}/processed_files/{file_name}")
  return df

In [13]:
def concat_files(trip="yellow", year=2019):
  starting_month = 1

  if year == 2019 and trip == "fhvhv":
    starting_month = 2

  ending_month = 12
  if year == 2024:
    ending_month = 8

  dfs = []
  for month in tqdm(range(starting_month, ending_month + 1)):
    path = f"{root_path}/processed_files/{trip}_tripdata_{year}-{str(month).zfill(2)}.parquet"
    df_fragment = pd.read_parquet(path)
    dfs.append(df_fragment)

  df = pd.concat(dfs, ignore_index = True)

  # Set license to yellow for yellow cab trips
  if trip == "yellow":
    df["license"] = "yellow"

  # Write files
  df.to_parquet(f"{root_path}/concatenated_files/{trip}_{year}.parquet")
  print(f"Wrote {root_path}/concatenated_files/{trip}_{year}.parquet")

## Iterate to Create Yearly Files

In [14]:
# for trip in ["fhvhv", "yellow"]:
#   for year in range(2019, 2025):
#     for month in range(12):
#       if trip == "fhvhv" and year == 2019 and month == 0:
#         # No file for FHVHV for January 2019.
#         continue
#       month = str(month + 1).zfill(2)
#       file_name = f"{trip}_tripdata_{year}-{month}.parquet"
#       print(f"Working on {file_name}")
#       process_file(trip, file_name)


In [15]:
# for trip in ["fhvhv", "yellow"]:
#   for year in range(2020, 2025):
#     concat_files(trip=trip, year=year)

## Aggregate Yellow and FHVHV Data to Make Hourly Totals

In [16]:
def aggregate_data(year=2024):
  destination_path = f"{root_path}/aggregated_hourly_files/{year}.parquet"
  yellow_df = pd.read_parquet(f"{root_path}/concatenated_files/yellow_{year}.parquet")
  fhvhv_df = pd.read_parquet(f"{root_path}/concatenated_files/fhvhv_{year}.parquet")
  df = pd.concat([yellow_df, fhvhv_df])

  # Limit to trips that took place in the appropriate year.
  df = df[df.pickup_datetime.dt.year == year]

  # Create trip duration column
  df["trip_duration"] = df.dropoff_datetime - df.pickup_datetime

  # Limit data to trip durations of under 2 hours.
  df = df[df.trip_duration.between(pd.Timedelta(minutes=1), pd.Timedelta(hours=2))]

  # Limit data to trip distances of under 10 miles.
  df = df[(df.trip_distance > 0) & (df.trip_distance <= 10)]

  # Create `date` and `hour` columns.
  df["datetime"] = pd.to_datetime(df.pickup_datetime)
  df = df.sort_values('datetime')
  df["datetime"] = df["datetime"].dt.floor('h')
  df["date"] = df.datetime.dt.date
  df["hour"] = df.datetime.dt.hour
  df.set_index('datetime', inplace=True)

  # Relabel license to give more helpful names.
  df["license"] = df["license"].replace(license_relabel)

  # Aggregate by hour and license
  license_agg_df = df.groupby(["date", "hour", "license"]).size().unstack(fill_value=0)

  agg_df = df.groupby(["date", "hour"]).agg(
      trip_count=("trip_duration", "count"),
      trip_duration_mean=("trip_duration", "mean"),
      trip_duration_median=("trip_duration", "median"),
      trip_duration_std_dev=("trip_duration", "std"),
      trip_duration_1Q=("trip_duration", lambda x: np.percentile(x, 25)),
      trip_duration_3Q=("trip_duration", lambda x: np.percentile(x, 75)),
      trip_distance_mean=("trip_distance", "mean"),
      trip_distance_median=("trip_distance", "median"),
      trip_distance_std_dev=("trip_distance", "std"),
      trip_distance_1Q=("trip_distance", lambda x: np.percentile(x, 25)),
      trip_distance_3Q=("trip_distance", lambda x: np.percentile(x, 75)),
      half_mile_trips=("trip_distance", lambda x: (x <= 0.5).sum()),
      one_mile_trips=("trip_distance", lambda x: ((x > 0.5) & (x <= 1)).sum()),
      two_mile_trips=("trip_distance", lambda x: ((x > 1) & (x <= 2)).sum()),
      three_mile_trips=("trip_distance", lambda x: ((x > 2) & (x <= 3)).sum()),
      five_mile_trips=("trip_distance", lambda x: ((x > 3) & (x <= 5)).sum())
  ).reset_index()
  merged_df = pd.merge(agg_df, license_agg_df, on=["date", "hour"], how="inner")

  merged_df.to_parquet(destination_path)

In [17]:
for year in tqdm(range(2019, 2025)):
  aggregate_data(year)

  0%|          | 0/6 [00:00<?, ?it/s]

In [18]:
dfs = []

for year in tqdm(range(2019, 2025)):
  source_path = f"{root_path}/aggregated_hourly_files/{year}.parquet"
  df_fragment = pd.read_parquet(source_path)
  dfs.append(df_fragment)

df = pd.concat(dfs, ignore_index=True)

df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['hour'].astype(str) + ':00')
df.set_index('datetime', inplace=True)

  0%|          | 0/6 [00:00<?, ?it/s]

In [20]:
df.to_parquet(f"{root_path}/complete_hourly.parquet")

In [21]:
df.columns

Index(['date', 'hour', 'trip_count', 'trip_duration_mean',
       'trip_duration_median', 'trip_duration_std_dev', 'trip_duration_1Q',
       'trip_duration_3Q', 'trip_distance_mean', 'trip_distance_median',
       'trip_distance_std_dev', 'trip_distance_1Q', 'trip_distance_3Q',
       'half_mile_trips', 'one_mile_trips', 'two_mile_trips',
       'three_mile_trips', 'five_mile_trips', 'juno', 'lyft', 'uber', 'via',
       'yellow'],
      dtype='object')